# Project: DaXiGua Agency

In [1]:
from State import State
from Movement_evaluation import evaluate_by_gravity
import numpy as np
from Ball import Ball
from Config import *
from Game import Game

## Test the movement evaluation

运动规则如下：

1. 所有球受重力影响下坠
2. 与墙壁和地板的碰撞无反弹
3. 有空气阻力，小球下落速度越大阻力越大
4. 所有球之间的碰撞有一定的速度损失，并非完全弹性碰撞
5. 两个同样大小的小球相撞后合成新的小球，新的小球半径增大50% （这一条需根据具体游戏规则再进行修改）

In [3]:
state_test_1 = State(screen_x, 
                     screen_y, 
                     [Ball(np.array([200, 555]), np.array([-10, -10.0]), ball_level = 1), 
                      Ball(np.array([100, 60]), np.array([20, 0]), ball_level = 2),
                      Ball(np.array([200, 60]), np.array([0, 0]), ball_level = 1),
                      Ball(np.array([300, 200]), np.array([0, 0]), ball_level = 0),
                      Ball(np.array([450, 30]), np.array([0, 0]), ball_level = 0)], 
                     end_line)

In [4]:
state_test_1.plot_state()

Position, x: 200, y: 555 
Velocity, vx: -10.0, vy: -10.0 
Level: 1,Radius: 40

Position, x: 100, y: 60 
Velocity, vx: 20, vy: 0 
Level: 2,Radius: 54

Position, x: 200, y: 60 
Velocity, vx: 0, vy: 0 
Level: 1,Radius: 40

Position, x: 300, y: 200 
Velocity, vx: 0, vy: 0 
Level: 0,Radius: 26

Position, x: 450, y: 30 
Velocity, vx: 0, vy: 0 
Level: 0,Radius: 26

----------------------------------------------------------------


### Initial State
<img src = "figures/sample_2.jpg" width="200"/>

In [8]:
state_test_1, obtained_score = evaluate_by_gravity(state_test_1, plot=False, verbose = True)
print(obtained_score)

Remove a cherry, obtain 2 score
Remove a orange, obtain 3 score
5


In [9]:
state_test_1.plot_state()

Position, x: 150.8419141866455, y: 59.5 
Velocity, vx: 0, vy: 0 
Level: 3,Radius: 59.5

Position, x: 300.0, y: 26.0 
Velocity, vx: 0, vy: 0 
Level: 0,Radius: 26

Position, x: 450.0, y: 26.0 
Velocity, vx: 0, vy: 0 
Level: 0,Radius: 26

----------------------------------------------------------------


### State after evaluation
<img src = "figures/sample_2_res.jpg" width="200"/>

## Test game

In [2]:
game = Game(screen_x, screen_y, end_line, balls_setting, max_random_ball_level)

In [3]:
# the inital state of the game
game.current_state.plot_state()

Position, x: 276.0, y: 884.5 
Velocity, vx: 0, vy: 0 
Level: 4,Radius: 76

----------------------------------------------------------------


### Step 0
<img src = "figures/sample_game_step0.jpg" width="200"/>

In [4]:
# We drop the ball from the middle of the screen
action = screen_x/2
game.next_step(action)
game.current_state.plot_state()

Position, x: 276.0, y: 76.0 
Velocity, vx: 0, vy: 0 
Level: 4,Radius: 76

Position, x: 276.0, y: 884.5 
Velocity, vx: 0, vy: 0 
Level: 4,Radius: 76

----------------------------------------------------------------


### Step 1
The green ball drops to the bottom, and a new blue ball is added
<img src = "figures/sample_game_step1.jpg" width="200"/>

In [5]:
action = screen_x/2
game.next_step(action)
game.current_state.plot_state()

Position, x: 276.0, y: 91.5 
Velocity, vx: 0, vy: 0 
Level: 5,Radius: 91.5

Position, x: 276.0, y: 884.5 
Velocity, vx: 0, vy: 0 
Level: 1,Radius: 40

----------------------------------------------------------------


In [7]:
print(game.current_reward)

5
